# Experimentos de Fourier

## Modelo de Fourier
## $$\dfrac{\partial T}{\partial t} = \alpha \dfrac{\partial^2 T}{\partial x^2}$$

## $$q = - \dfrac{\partial T}{\partial x}$$

## Con las variables causales
### $A = \dfrac{\partial T}{\partial t}, \quad B = \dfrac{\partial^2 T}{\partial x^2}$
### $C = q, \quad D  = \dfrac{\partial T}{\partial x}, \quad E = T$

La ecuación de transporte esta dada en su forma adimensional por medio de las siguientes variables adimensionales y parámetros:
* $T$ es la temperatura
* $t$ es el tiempo
* $x$ es la dirección espacial x
* $q$ es el flujo de calor
* El parámetro $\alpha$ es el número de Fourier $\alpha = D_0 t_0 L^{-2}$
* Con $t_c$ es el tiempo característico, $D$ es la difusividad térmica del material y $L$ es la longitud del material

Este problema en particular se va a resolver para dos casos a las condiciones de frontera:

Determinista
* $\quad T(x = 1, t) = 1 \quad$ $T(x = 0, t) = \mu_\epsilon$

Estocástico
* $\quad T(x = 1, t) = 1 \quad$ $T(x = 0, t) = \epsilon(t)$ 

Con $\epsilon(t)$ ruido blanco Gaussiano y $\mu_\epsilon$ su promedio

Y la condición inicial para ambos casos
* $T(x, t = 0) = 1$

### Importación de bibliotecas y módulos

In [1]:
from IPython import display
import ipywidgets
import numpy as np               
from numba import jit
import modulo_fourier as mf

## Función de simulación
Dependiendo de la frontera, se tiene la simulacion determinista o estocastica

In [2]:
@jit(nopython = True)
def simulacion(frontera, perfil_analitico, variacion_inicial ):
    global nx, nt, alfa, tau, dx, dt, xpi
    # Condicion inicial
    temperatura = perfil_analitico + variacion_inicial
    # Inicializacion de las series de tiempo
    serie_a = np.empty(nt)
    serie_b = np.empty(nt)
    serie_c = np.empty(nt)
    serie_d = np.empty(nt)
    serie_e = np.empty(nt)
    # Proceso de la simulacion
    for n in range(nt):
        # Frontera. Al instante n se le asigna el valor correspondiente a la frontera
        temperatura[0] = frontera[n]
        # Calculo de la Temperatura
        temperatura = mf.rungekutta(temperatura, alfa, dx, dt)
        # Calculo del flujo de calor
        grad_temperatura = mf.ddx(temperatura, dx, full = True)
        flujocalor = mf.calc_q(grad_temperatura)
        # Calculo del miembros de la ecuacion de transporte
        b = mf.d2dx2(temperatura, dx, full = True)
        a = alfa * b
        # Captura de las series de tiempo
        serie_a[n] = a[xpi]
        serie_b[n] = b[xpi]
        serie_c[n] = flujocalor[xpi]
        serie_d[n] = grad_temperatura[xpi]
        serie_e[n] = temperatura[xpi]
    # Se devuelven las series de tiempo de la simulacion    
    return serie_a, serie_b, serie_c, serie_d, serie_e

## Función de fluctuación
Construcción de la fluctuación de una variable del modelo

In [3]:
@jit(nopython = True)
def fluctuacion(determinista, estocastico):
    return estocastico - determinista

## Archivo del ensamble de realizaciones de la frontera

In [4]:
ensamble_fronteras = np.loadtxt("datos/base/fronteras.csv", delimiter = ",")
ensamble_fronteras.shape

(128, 40001)

## Archivo de la condición inicial de la Temperatura

In [5]:
condicion_inicial = np.loadtxt("datos/base/variacion_inicial.csv")
condicion_inicial.shape

(128, 161)

## Dominio espacial

La solución numérica del problema se presenta solo en una dimensión espacial

In [6]:
x = np.loadtxt("datos/base/espacio_simulacion.csv")  # Vector del dominio espacial
nx = x.size                                          # Numero de puntos en el espacio
Lx = x[-1]                                           # Longitud del dominio espacial
dx = x[1] - x[0]                                     # Tamaño de paso en el espacio
dx

0.00625

## Dominio temporal

El tiempo que se pretende dar solución al sistema de ecuaciones

In [7]:
t = np.loadtxt("datos/base/tiempo_simulacion.csv")  # Vector del dominio temporal, contiene nt+1 puntos incluyendo el 0
nt = ensamble_fronteras.shape[1]                    # Numero de pasos en el tiempo
Lt = t[-1]                                          # Longitud del dominio temporal
dt = t[1] - t[0]                                    # Tamaño de paso en el tiempo, 
dt

2.5e-05

## Parámetros de la simulación

Para los arreglos

In [8]:
xpi = int(nx / 5)                 # Punto de interes donde se capturan las series de tiempo
nr = ensamble_fronteras.shape[0]  # Numero de realizaciones del ensamble
x[xpi]

0.2

Valores termodinámicos

In [9]:
difusividad = 1.0                         # Valor de referencia de la difusividad
t_c = (Lx * Lx) / difusividad             # Tiempo caracteriztico
num_alfa = difusividad * t_c / (Lx * Lx)  # Valor del parametro alfa
t_c

1.0

Número de Fourier

In [10]:
alfa = num_alfa * np.ones(nx)  # Vector/arreglo alfa
alfa[0], alfa[-1] = 0, 0       # Extremos del vector/arreglo alfa

## Inicialización

In [11]:
fluc_a = np.empty((nr, nt))
fluc_b = np.empty((nr, nt))
fluc_c = np.empty((nr, nt))
fluc_d = np.empty((nr, nt))
fluc_e = np.empty((nr, nt))
serie_a = np.empty((nr, nt))
serie_b = np.empty((nr, nt))
serie_c = np.empty((nr, nt))
serie_d = np.empty((nr, nt))
serie_e = np.empty((nr, nt))

## Proceso principal

In [12]:
# Barra de progreso
progreso = ipywidgets.FloatProgress(
    value = 0,
    min = 0,
    max = nr, 
    description = 'Progreso:')
display.display(progreso)

# Proceso principal
progreso.value = 0
for r in range(nr):
    # Fronteras
    frontera_est = ensamble_fronteras[r]
    frontera_det = np.mean(frontera_est) * np.ones(nt)
    # Solucion analitica de la realizacion
    mu_frontera = np.mean(frontera_est) - 1.0
    perfil_analitico = 1.0 + mu_frontera * (1.0 - x) 
    # Condicion inicial
    variacion_inicial = condicion_inicial[r]
    # Simulaciones
    a_est, b_est, c_est, d_est, e_est = simulacion(frontera_est, perfil_analitico, variacion_inicial)
    a_det, b_det, c_det, d_det, e_det = simulacion(frontera_det, perfil_analitico, variacion_inicial)
    # Fluctuaciones
    fluc_a[r] = fluctuacion(a_det, a_est)
    fluc_b[r] = fluctuacion(b_det, b_est)
    fluc_c[r] = fluctuacion(c_det, c_est)
    fluc_d[r] = fluctuacion(d_det, d_est)
    fluc_e[r] = fluctuacion(e_det, e_est)
    # Series
    serie_a[r] = a_det
    serie_b[r] = b_det
    serie_c[r] = c_det
    serie_d[r] = d_det
    serie_e[r] = e_det
    # Progreso del trabajo
    progreso.value = r + 1

FloatProgress(value=0.0, description='Progreso:', max=128.0)

## Datos

In [13]:
variables = ["a", "b", "c", "d", "e"]
series = [serie_a, serie_b, serie_c, serie_d, serie_e]
fluctuaciones = [fluc_a, fluc_b, fluc_c, fluc_d, fluc_e]

datos_series = dict(zip(variables, series))
datos_fluctuaciones = dict(zip(variables, fluctuaciones))

## Archivos

In [14]:
ruta = "datos/experimentos_fourier/"
for d in datos_series.keys():
    np.savetxt(ruta + "serie_" + d + ".csv", datos_series[d], delimiter = ",")
    np.savetxt(ruta + "fluctuacion_" + d + ".csv", datos_fluctuaciones[d], delimiter = ",")